# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [131]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [70]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kill4\AppData\Roaming\nltk_data...


True

In [98]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kill4\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [147]:
df = pd.read_csv('ru_recipes_sample.csv'); df
s1 = pd.Series(df.description.values)
arr2 = s1.values.copy()
s1 = s1.apply(lambda x: word_tokenize(x))
arr = np.array(s1)

words = []
for lst in arr:
    for word in lst:
        if word.isalpha() and word not in words:
            words.append(word)
arr

array([list(['Этот', 'коктейль', 'готовлю', 'из', 'замороженной', 'клубники', '.', 'Если', 'клубника', 'свежая', ',', 'то', 'добавляю', 'перепелиное', 'яйцо', ',', 'благодаря', 'этому', 'коктейль', 'получается', 'устойчиво', 'густым', '.']),
       list(['Быстро', 'и', 'вкусно', '.']),
       list(['Сытный', ',', 'овощной', 'салатик', ',', 'пальчики', 'оближете', '.']),
       ...,
       list(['Мое', 'любимое', 'блюдо', 'лазанья', '.', 'Но', 'кушать', 'только', 'фарш', 'уже', 'поднадоело', ',', 'потому', 'добавляю', 'в', 'красный', 'соус', 'овощи', '.', 'Соус', 'видоизменен', ',', 'потому', 'это', 'уже', 'будет', 'не', 'тот', 'самый', 'Болоньез', '.', 'Но', 'подойдет', 'он', 'ко', 'всему', '.']),
       list(['Прошлым', 'летом', 'варила', 'варенье', 'из', 'одуванчиков', 'по', 'рецептам', 'нашего', 'сайта', ',', 'а', 'сейчас', 'попробовала', 'новый', 'вариант', '.', 'Благодарю', 'Сергея', 'Самойлова', ',', 'его', 'рецепт', 'взяла', 'за', 'основу', ',', 'немного', 'изменив', 'приготовле

In [148]:
len(words)
words = np.array(words)

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [149]:
from nltk.metrics.distance import edit_distance

In [150]:
s1 = pd.Series(words)
df = pd.DataFrame(data = {'1': s1.sample(n = 5).values, '2' : s1.sample(n = 5).values})
array = []
for i in range(5):
    array.append(edit_distance(df['1'][i], df['2'][i]))
df['distance'] = array; df

,1,2,distance
0,маринованных,кольца,11
1,понадобиться,местах,11
2,Русская,треугольники,11
3,Выглядит,Крабовые,8
4,финиками,годы,8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [151]:
def knn(word:str, k:int):
    s1 = pd.Series(words)
    distances = s1.apply(lambda x: edit_distance(word, x, substitution_cost=2))
    indxs = np.argsort(np.array(distances))[:k]
    return words[indxs]

In [152]:
knn('булка', 3)

array(['булка', 'белка', 'булку'], dtype='<U21')

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [153]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 

In [154]:
snb_stemmer_ru = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


In [155]:
df1 = pd.DataFrame(data = words, columns=['word']);
df1['stemmed_word'] = df1['word'].apply(lambda x: snb_stemmer_ru.stem(x))
df1['normalized_word'] = df1['word'].apply(lambda x: morph.parse(x)[0].normal_form)
df1 = df1.set_index('word')
df1

,stemmed_word,normalized_word
word,,
Этот,этот,этот
коктейль,коктейл,коктейль
готовлю,готовл,готовить
из,из,из
замороженной,заморожен,заморозить
...,...,...
сбывается,сбыва,сбываться
подружим,подруж,подружить
одинаковых,одинаков,одинаковый


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [171]:
arr2 = []
for lst in arr:
    for word in lst:
        arr2.append(word)
before_obr = arr2.copy()
len(arr2)

128822

In [172]:
ru_stop_words = stopwords.words('russian')
p = re.compile('^[а-яА-ЯёЁ*&]')
arr2 = [i for i in arr2 if p.search(i)]
ru_stop_words = stopwords.words('russian')
a = [w.lower() for w in arr2 if w.lower() not in ru_stop_words]
len(a)

69349

In [173]:
print(f'Доля стоп-слов: {(len(arr2) - len(a))/ len(arr2)}')

Доля стоп-слов: 0.32368831675443727


In [185]:
before_obr = [i for i in before_obr if p.search(i)]
do_udal = pd.Series(before_obr)
posle = pd.Series(a)
print(f'Топ-10 до удаления стоп-слов:\n{do_udal.value_counts()[:10]}')
print()
print(f'Топ-10 после удаления стоп-слов:\n{posle.value_counts()[:10]}')

Топ-10 до удаления стоп-слов:
и        4822
в        2304
с        1850
на       1504
не       1427
очень    1123
из        977
я         752
что       732
для       686
dtype: int64

Топ-10 после удаления стоп-слов:
очень          1608
рецепт          869
это             733
блюдо           525
вкусный         461
просто          436
вкусно          378
приготовить     344
вкус            323
салат           313
dtype: int64


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).